In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#Bandits

bandits = [0.2, 0, -.2, -5]
num_bandits = len(bandits)

def pullBandit(bandit):
    result = np.random.randn(1)
    return 1 if result > bandit else -1

In [3]:
sess = tf.InteractiveSession()

In [4]:
#These two lines established the feed-forward part of the network. This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#The next six lines establish the training proceedure. We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [5]:
tf.global_variables_initializer().run()

In [9]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.


# Launch the tensorflow graph
i = 0
while i < total_episodes:

    #Choose either a random action or one from our network.
    if np.random.rand(1) < e:
        action = np.random.randint(num_bandits)
    else:
        action = sess.run(chosen_action)

    reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.

    #Update the network.
    _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})

    #Update our running tally of scores.
    total_reward[action] += reward
    if i % 50 == 0:
        print("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
    i+=1
print("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 bandits: [0. 0. 0. 1.]
Running reward for the 4 bandits: [ 1.  1.  0. 47.]
Running reward for the 4 bandits: [ 1.  1.  0. 95.]
Running reward for the 4 bandits: [  2.   1.  -1. 143.]
Running reward for the 4 bandits: [  2.   0.   1. 188.]
Running reward for the 4 bandits: [  1.   0.   1. 237.]
Running reward for the 4 bandits: [  2.  -1.   2. 282.]
Running reward for the 4 bandits: [  3.  -2.   5. 327.]
Running reward for the 4 bandits: [  3.  -3.   4. 371.]
Running reward for the 4 bandits: [  2.  -1.   3. 415.]
Running reward for the 4 bandits: [  0.  -1.   5. 459.]
Running reward for the 4 bandits: [ -1.   0.   5. 505.]
Running reward for the 4 bandits: [ -2.  -1.   6. 552.]
Running reward for the 4 bandits: [ -3.  -1.   6. 597.]
Running reward for the 4 bandits: [ -2.  -2.   6. 641.]
Running reward for the 4 bandits: [ -3.  -2.   7. 689.]
Running reward for the 4 bandits: [ -5.  -2.   8. 734.]
Running reward for the 4 bandits: [ -6.  -2.   9. 778.]
Running 